# Fix the Version of Python Library Scikit-learn

This notebook ensures the AI-Lab using the same version of python library `scikit-learn` as the one used by the <a href="https://docs.exasol.com/db/latest/database_concepts/udf_scripts/adding_new_packages_script_languages.htm" target="_blank" rel="noopener">Script Language Container (SLC)</a> inside Exasol database.

## Rationale

Using identical versions is required when transferring the sklearn model from the AI-Lab to the database SLC.

The AI-Lab serializes the sklearn model and uploads it into the database SLC. The SLC can only _deserialize_ the model if using the same version of the `scikit-learn` library. The specific version of the library used by the SLC depends on the release version of the database and cannot be controlled by the AI-Lab. Claiming to be intelligent, the AI-Lab adapts its own version of the library if required.

## Open Secure Configuration Storage

In [23]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

Output()

Box(children=(Box(children=(Label(value='Configuration Store', layout=Layout(border_bottom='solid 1px', border…

## Detect Versions and Update Installation if Required

In [31]:
import textwrap
import sklearn
from exasol.nb_connector.connections import open_pyexasol_connection

# Create script to test the model
sql = textwrap.dedent("""
CREATE OR REPLACE PYTHON3 SCALAR SCRIPT {schema!q}.detect_skikit_learn_version() RETURNS VARCHAR(100) AS
import sklearn
def run(ctx):
    return sklearn.__version__ 
/
""")

with open_pyexasol_connection(ai_lab_config, compression=True) as conn:
    query_params={'schema': ai_lab_config.db_schema}
    conn.execute(sql, query_params)
    result = conn.execute("select {schema!q}.detect_skikit_learn_version()", query_params).fetchone()
    slc_version = result[0]

# slc_version = "1.4.2"

my_version = sklearn.__version__

if slc_version == my_version:
    print(f"AI-Lab scikit-learn version {my_version} is identical to that of the SLC.\nNothing to do.")
else:
    print(f"AI-Lab scikit-learn version {my_version} differs from SLC.\nInstalling version {slc_version} ...")
    %pip install "scikit_learn=={slc_version}"
    # print(f'%pip install f"scikit_learn=={slc_version}"')
    # %pip show scikit_learn

AI-Lab scikit-learn version 1.0.2 differs from SLC.
Installing version 1.4.2 ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 6.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: scikit_learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Note: you may need to restart the kernel to use updated packages.
